# Mapping stellar surfaces with `starry`

In this tutorial, we'll create a random starspot map and generate a corresponding light curve as TESS might see it. We'll then use `starry` to invert this light curves and infer what the star actually looks like.

**NOTE:** *I assume some basic familiarity with `starry`. If you are new to the code, check out the [tutorials page](https://rodluger.github.io/starry/tutorials.html).*

## Imports & setup

In [ ]:
%matplotlib inline

In [ ]:
%run workshop_utils/notebook_setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

I added some handy utils to the `workshop_utils` module (in the same directory as this notebook). Let's import them:

In [ ]:
from workshop_utils import animate, set_random, MAP, orbit

## Instantiate a `starry` map

Let's instantiate a 10th degree spherical harmonic map. We'll give it an inclination of 60 degrees with respect to the observer.

In [ ]:
# Highest spherical harmonic degree of the map
lmax = 10

# The total number of spherical harmonic coefficients
N = (lmax + 1) ** 2

# Instantiate the star, a `Map` object
map = starry.Map(lmax)

# Specify the axis of rotation of the star with some
# simple trig
inc = 60 * (np.pi / 180)
axis = [0, np.sin(inc), np.cos(inc)]
map.axis = axis

Let's give the map a random set of spherical harmonic coefficients. At each degree $l$, we'll draw the coefficients from a zero-mean gaussian with a standard deviation $\alpha$ that decreases with $l$: 

$\alpha(l) \propto \mathrm{e}^{-\frac{l^2}{2\sigma_l^2}}$.

In [ ]:
# Set the randomizer seed (change this for completely different maps!)
np.random.seed(41)
set_random(map, sigma_l=2.5, norm=0.01)

We need to be a little careful at this point, since a purely random map can actually have a *negative* intensity at certain points on the surface. The scale of the features is `0.01` (the `norm` constant in the call to `set_random` above), so we can enforce positive intensity everywhere by setting the baseline to be unity:

In [ ]:
map[0,0] = 1.0

OK, now let's plot the power spectrum of the map to see what we're dealing with. Recall that the power at a given degree $l$ is just the sum of the squares of the spherical harmonic coefficients of that degree.

In [ ]:
power_true = [np.sum(map[l, :] ** 2) for l in range(lmax + 1)]
l = np.arange(lmax + 1)
plt.plot(l, power_true, "o-", label="Measured")
plt.yscale("log")
plt.xlabel("Spherical harmonic degree")
plt.ylabel("Power");

Cool. On to the fun part: let's animate the map as it rotates. I implented a handy routine called `animate` in the `workshop_utils` module, so let's use that.

In [ ]:
animate(map)

## ★ Exercise 1

Sample the power spectrum a few times and get a feel for how different maps with the same power spectrum can look. Then, vary the scale of the power spectrum to see how the scale of the surface features changes.

## Generate a synthetic TESS light curve

Now let's generate a simulated light curve. In the spirit of TESS, let's produce a light curve sampled every 2 minutes for 27 days. We'll give the star a period of **9.67 days**. We'll also add some photon noise to make things fun.

In [ ]:
# Generate the light curve using the `flux` method
time = np.arange(0, 27, 1.0 / (24 * 30))
per = 9.67
theta = 360. / per * time
flux_true = map.flux(theta=theta)

# Add noise
flux_err = 0.05 * np.std(flux_true)
flux = flux_true + flux_err * np.random.randn(len(flux_true))

Plot the data:

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 5))
plt.plot(time, flux, 'k.', ms=2, alpha=0.3, label="Observed")
plt.plot(time, flux_true, label="True")
plt.legend(numpoints=3)
plt.xlabel("Time (days)")
plt.ylabel("Flux");

## ★ Exercise 2

Produce light curves for your different samples in Exercise 1 and get a feel for all the different light curves you can get from similarly-behaved maps.

## Now let's try to recover the map!

Here we will attempt to "invert" the light curve and fit for the actual map that generated it. In general, we would have to solve for many things: the period of the star, the inclination, the power spectrum, as well as every single spherical harmonic coefficient. But *for simplicity*, let's assume we know (or have good constraints on) the period and inclination of the star.

Even with the period and inclination fixed, we still have a **lot** of parameters to solve for. A degree $l=10$ map has $(l + 1)^2 = 121$ spherical harmonic coefficients. We could imagine dumping the problem into something like `emcee` and crunching away with MCMC. But with 121 coefficients, we'd need *at least* 300 or so walkers, and it would probably take us (I'm guessing) on the order of 50,000 chain iterations to get a decent estimate of the posterior. That's 15 *million* function calls. The `starry` algorithm is pretty fast, but for a light curve this long, a single evaluation takes about 0.1 seconds. All together, that's about 2 million seconds of run time, or **just over 23 CPU days**.

**Yikes.**

Fortunately, there's a **MUCH** faster way of doing this.

### The power of a linear model

If you read our [paper](http://adsabs.harvard.edu/abs/2019AJ....157...64L), recall that the `starry` model for a series of flux measurements $\mathbf{f}$ is given by

$\mathbf{f} = \mathbf{S}^\mathrm{T} \mathbf{A} \ \mathbf{R} \ \mathbf{R} \ \mathbf{y}$

where $\mathbf{y}$ is the vector of spherical harmonic coefficients and all the other variables are matrices determined by the geometry of the problem. So if we define

$\mathcal{A} \equiv \mathbf{S}^\mathrm{T} \mathbf{A} \ \mathbf{R} \ \mathbf{R}$

the computation of the light curve model is a **purely linear problem**:

$\mathbf{f} = \mathcal{A} \ \mathbf{y}$

The equation above tells us how to compute the flux $\mathbf{f}$ given a set of spherical harmonic coefficients $\mathbf{y}$. But we are interested in the *opposite* problem: given $\mathbf{f}$, how do we compute $\mathbf{y}$?

Since everything is linear and are errors are gaussian (by construction!), the **solution is analytic!** If you're interested, check out section 2.1 of [this paper](http://adsabs.harvard.edu/abs/2018AJ....156...99L) for the derivation. But for now, let's just take the result for granted:

$\hat{\mathbf{y}} = \left(\mathcal{A}^\mathrm{T} \mathcal{A} + \sigma^2\mathbf{\Lambda}^{-1}\right)^{-1}\mathcal{A}^\mathrm{T}\mathbf{f}$

$\hat{\Sigma} = \sigma^2\left(\mathcal{A}^\mathrm{T} \mathcal{A} + \sigma^2\mathbf{\Lambda}^{-1}\right)^{-1}$

where $\hat{\mathbf{y}}$ is the MAP (maximum a posteriori) estimate of the map coefficients, and $\hat{\Sigma}$ is the covariance matrix of that estimate. I've assumed that our light curve $\mathbf{f}$ has homoscedastic noise (all data points are uncorrelated and have the same error $\sigma$). Also, note that since we're doing Bayesian inference, we need a prior on the map coefficients! That's what the matrix $\mathbf{\Lambda}$ is: it's the prior covariance of the coefficients.

Note that the equations above give us the full posterior over map coefficients. Since the problem is linear, all the posteriors are gaussian, and are completely described by the mean and the variance. So if we can solve those two equations, **we are done!** No need for MCMC.

## ★ Exercise 3

Derive the equations presented above for the MAP estimate and its variance. A good starting point is section 2.1 of [this paper](http://adsabs.harvard.edu/abs/2018AJ....156...99L). *Hint*: The MAP solution is obtained by maximizing the log posterior, so you'll want to differentiate it with respect to $\mathbf{y}$ and set the expression to zero. And the variance of a gaussian is the inverse of its Hessian, so you'll need to take the *second* derivative.

### Let's do it!

**NOTE:** *Version 1.0.0 of `starry` will implement the linear solver as the `MAP` method of a `Map` instance. But for now, I implemented this method in the `workshop_utils` module, so we can skip all the tedious linear algebra!*

The one bit of work we need to do is figure out our prior. If we happen to *know* the power spectrum the map was generated from (which we do!), the variance of each coefficient is just:

In [ ]:
lam = np.array([power_true[l] for l in range(lmax + 1) for m in range(-l, l + 1)])
lam[0] = np.inf

In our case, $\mathbf{\Lambda}$ is a diagonal matrix: the coefficients are all uncorrelated. In general, we will not know the actual power spectrum of the map, but in principle it is something we could **fit** for!

Anyways, let's get to it. We'll time the operation just for fun. Recall that the time to beat is **23 days.**

In [ ]:
%%time
yhat, yvar = MAP(lmax, flux, flux_err, lam=lam, axis=axis, theta=theta)

## Visualize the results

Let's load the MAP coefficients into the `map_ml` (for maximum likelihood) object and compute the model:

In [ ]:
map_ml = starry.Map(lmax)
map_ml.axis = axis
map_ml[:, :] = yhat
model_ml = np.array(map_ml.flux(theta=theta))

Let's also instantiate 50 other maps and populate them with random draws from the posterior (which is easy, since we have a covariance matrix). We'll put these maps in the list `map_draw`. We'll also compute the model for each one.

In [ ]:
nsamp = 50
y_draw = np.empty((nsamp, map.N))
model_draw = np.empty((nsamp, len(time)))
np.random.seed(43)
map_draw = starry.Map(lmax)
map_draw.axis = map.axis
for i in range(nsamp):
    y_draw[i] = np.random.multivariate_normal(yhat, yvar)
    map_draw[:, :] = y_draw[i]
    model_draw[i] = np.array(map_draw.flux(theta=theta))

Now we plot the MAP light curve alongside the random posterior draws superimposed on the data:

In [ ]:
plt.plot(time, flux, 'k.', ms=2, alpha=0.3, label="observed")
for i in range(nsamp):
    plt.plot(time, model_draw[i], 'C1', alpha=0.1, label="sample" if i == 0 else None)
plt.plot(time, model_ml, 'C1', label="MAP")
plt.ylabel("Flux")
plt.xlabel("Time")
plt.legend(numpoints=5, fontsize=12);

Here's what the MAP star looks like next to the true star:

In [ ]:
animate(map, map_ml, titles=["True", "MAP"])

And here's four random samples:

In [ ]:
map_draws = []
for i in range(4):
    map_draws.append(starry.Map(lmax))
    map_draws[-1].axis = map.axis
    map_draws[-1][:, :] = y_draw[i]
animate(map, map_ml, *map_draws, res=75, vmin=0.28, vmax=0.36,
        titles=["True", "MAP", "Sample", "Sample", "Sample", "Sample"])

If we plot the power spectrum of the inferred maps next to the power spectrum of the true map, we see they agree pretty well! But for $l =$ 3, 5, 7, and 9, the MAP power is *orders of magnitude* smaller. Why is that?

In [ ]:
power_ml = [np.sum(map_ml[l,:] ** 2) for l in range(lmax + 1)]
l = np.arange(lmax + 1)
plt.plot(l, power_true, label="Actual")
plt.plot(l, power_ml, 'o', label="MAP")
for i in range(50):
    map_draw[:, :] = y_draw[i]
    power_draw = [np.sum(map_draw[l,:] ** 2) for l in range(lmax + 1)]
    plt.plot(l, power_draw, color="C1", alpha=0.1, label="Sample" if i == 0 else None, zorder=-10)
plt.yscale("log")
plt.xlabel("Degree")
plt.ylabel("Power")
plt.legend();

## ★ Exercise 4

Increase the noise in the light curve and investigate how our MAP estimate and the posterior samples vary.

## ★ Exercise 5

Above, we assumed we knew the exact values of the period and inclination, which is rarely ever (actually *never*) the case. Think about how one might adapt this approach to the general case where the period and the inclination must be inferred from the data.

## Let's add some occultations to the mix

Let's add a small companion to the star (a planet or a low mass star) and generate a light curve that also includes occultations. For simplicity we'll ignore the secondary eclipse (the companion passing behind the star). We'll also put the companion on a very short period, close-in orbit so that we observe several occultations.

The `worskop_utils` module implements a simple `orbit` function that returns the cartesian position of a body as a function of time in a Keplerian orbit. We'll use that to specify the trajectory of the occultor.

**NOTE:** *In the beta version of `starry`, we'd instantiate `Primary`, `Secondary`, and `System` objects to compute the Keplerian solution and the full light curve. These are deprecated in version `1.0.0`, which will take advantage of the awesome functionality in the `exoplanet` package. Stay tuned.*

In [ ]:
# Occultor radius (relative to star)
ro = 0.1

# Cartesian position as a function of time
xo, yo, zo = orbit(time, porb=2.954, a=2.1, inc=88.5, tref=1.0)

# When the occultor is behind the star, artifically put it 
# at infinity so `starry` doesn't compute an occultation
xo[zo < 0] = -np.inf

# Compute the flux without and with the occultations
flux_true_no_occ = map.flux(theta=theta)
flux_true = map.flux(theta=theta, xo=xo, yo=yo, ro=ro)

# Add noise
flux_err = 0.1 * np.std(flux_true)
flux = flux_true + flux_err * np.random.randn(len(flux_true))

Let's plot our new light curve:

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 5))
plt.plot(time, flux, 'k.', ms=2, alpha=0.3, label="Observed")
plt.plot(time, flux_true_no_occ, color="C0", ls="--", label="No occultation")
plt.plot(time, flux_true, color="C0", label="True")
plt.legend(numpoints=3, fontsize=12)
plt.xlabel("Time (days)")
plt.ylabel("Flux");

As before, let's invert the light curve and compute the posterior over maps. As before, the time to beat is about **23 days**:

In [ ]:
%%time
yhat, yvar = MAP(lmax, flux, flux_err, lam=lam, axis=axis, theta=theta, xo=xo, yo=yo, ro=ro)

Now instantiate a new `starry` map and compute the MAP light curve:

In [ ]:
map_ml_occ = starry.Map(lmax)
map_ml_occ.axis = axis
map_ml_occ[:, :] = yhat
model_ml_occ = np.array(map_ml_occ.flux(theta=theta, xo=xo, yo=yo, ro=ro))

Compute 50 posterior samples as before:

In [ ]:
nsamp = 50
y_draw = np.empty((nsamp, map.N))
model_draw = np.empty((nsamp, len(time)))
np.random.seed(43)
map_draw = starry.Map(lmax)
map_draw.axis = map.axis
for i in range(nsamp):
    y_draw[i] = np.random.multivariate_normal(yhat, yvar)
    map_draw[:, :] = y_draw[i]
    model_draw[i] = np.array(map_draw.flux(theta=theta, xo=xo, yo=yo, ro=ro))

Plot the models:

In [ ]:
plt.plot(time, flux, 'k.', ms=2, alpha=0.3, label="observed")
for i in range(nsamp):
    plt.plot(time, model_draw[i], 'C1', alpha=0.1, label="sample" if i == 0 else None)
plt.plot(time, model_ml_occ, 'C1', label="MAP")
plt.ylabel("Flux")
plt.xlabel("Time")
plt.legend(numpoints=5, fontsize=12);

Take a look at the power spectrum of the inferred maps:

In [ ]:
power_ml = [np.sum(map_ml_occ[l,:] ** 2) for l in range(lmax + 1)]
l = np.arange(lmax + 1)
plt.plot(l, power_true, label="Actual")
plt.plot(l, power_ml, 'o', label="MAP")
for i in range(50):
    map_draw[:, :] = y_draw[i]
    power_draw = [np.sum(map_draw[l,:] ** 2) for l in range(lmax + 1)]
    plt.plot(l, power_draw, color="C1", alpha=0.1, label="Sample" if i == 0 else None, zorder=-10)
plt.yscale("log")
plt.xlabel("Degree")
plt.ylabel("Power")
plt.legend();

Finally, let's compare the true map, the rotation only solution, and the rotation + occultation solution:

In [ ]:
animate(map, map_ml, map_ml_occ, titles=["True", "Rotation only", "Rotation + Occultation"])

## ★ Exercise 6

Plot some posterior map samples, and comment on how they differ from the rotation only case. 

## We're done!

That's it for now! Check out the [documentation](https://rodluger.github.io/starry/) for more examples and tutorials, and remember to keep an eye on the [dev branch](https://github.com/rodluger/starry/tree/dev) for the upcoming release of `starry 1.0.0`. Feel free to [email me](mailto:rluger@flatironinstitute.org) with questions/suggestions and ideas on how to improve `starry`!